In [1]:
#Read Data From CSV file
import pandas as pd
df=pd.read_csv('NEWDATATABLE.csv', sep=',',header=None)
ratings = df.values

In [2]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [3]:
def train_test_generation(ratings):
    #Cretae train and test matrix
    test = np.ones(ratings.shape) * 99 #24983*100 and initzalize all be unknown 99 and ratings.shape[0] return 24983
    train = ratings.copy()
    #Make all unknown numbers 99 be 0
    train = train - np.ones(ratings.shape) * 99
    for user in xrange(ratings.shape[0]):
        #Randomly pick 10 ratings from each user to be in test case (include unknown ratings)
        test_ratings = np.random.choice(ratings[user, :].nonzero()[0], size = 20, replace = False)
        #Set chosen be zero in training
        train [user, test_ratings] = 0.
        #Assign chosen rating to test case
        test[user, test_ratings] = ratings[user, test_ratings]
    
    train = train + np.ones(ratings.shape) * 99
    #Make sure train and test are disjoint 
    #assert(np.all((train * test) == 0))
    return train, test

In [4]:
#Call the function and then assign train and test
train, test = train_test_generation(ratings)
trainMAE = train.copy()
#Masked for CF and not masked for SVD
train = np.ma.masked_where(train == 99, train)
test = np.ma.masked_where(test == 99, test)

In [8]:
#Compute MAE in training and test
def get_mae(pred, actual):
    mae_sum = 0
    count = 0
    for indexI in range(24983):
        for indexJ in range(100):
            if actual[indexI][indexJ] != 99:
                mae_sum = mae_sum + np.abs(pred[indexI][indexJ] - actual[indexI][indexJ])
                count = count + 1    
    mae_sum = float(mae_sum) / count
    return mae_sum

In [22]:
def SGD_factorization(train, K, steps=150, alpha=0.0001, beta=0.02):
    N_user, N_item = train.shape
    #U and V are latent vectors
    U = np.random.rand(N_user,K)
    V = np.random.rand(N_item,K).T
    
    #Run 10 steps
    for step in xrange(steps):
        for i in xrange(N_user):
            for j in xrange(N_item):
                if train[i,j] > -10 and train[i,j] < 10:
                    eij = train[i][j] - np.dot(U[i,:],V[:,j])
                    U[i] = U[i] + alpha * (2 * eij * V[:,j] - beta * U[i])
                    V[:,j] = V[:,j] + alpha * (2 * eij * U[i] - beta * V[:,j])
    nR = np.dot(U,V)
    return nR

In [ ]:
K = 10
prediction_SGD = SGD_factorization(train, K=10, steps=150, alpha=0.0001)
print 'SGD Train MAE: ' + str(get_mae(prediction_SGD, train))
print 'SGD Test MAE: ' + str(get_mae(prediction_SGD, test))